Import some modules, and set up this notebook to read files correctly.

In [1]:
import os
os.chdir('../..')
from pipelines.util import *
import pandas as pd

Read the data files into a pandas dataframe

In [2]:
data = pd.read_excel(os.path.join(WDIR, 'true-north/OITrue North data June 2024[32].xlsx'))
# additional_members = pd.read_excel(os.path.join(WDIR, 'true-north/true north_additional_members_list_19_06_24.xlsx'))

In [3]:
# additional_members['Create Date'] = '2024-06-20 00:00:00'

In [4]:
# data = pd.concat([hubspot_export, additional_members]).reset_index(drop=True)
# data = hubspot_export

The dates should all start in May 2023 or later.

In [5]:
data['Create Date'] = pd.to_datetime(data['Create Date'])
start_date = pd.Timestamp('2023-05-01')
data.loc[data['Create Date'] < start_date, 'Create Date'] = start_date

Add a "last_updated" column which uses the 'last activity date' column if it exists, otherwise the create date.

In [6]:
# Currently have both create date (when the person filled the questionnaire for the first time) and the "last_updated"
# date, which is the "Last Activity Date" iff it exists, otherwise it is the "Create Date"
data['last_updated'] = data['Last Activity Date'].combine_first(data['Create Date'])
data.columns.to_list()

['First Name',
 'Last Name',
 'Create Date',
 'Last Activity Date',
 'How many people work in your organisation?',
 'What sector(s) does your organisation operate in? [Other]',
 'What sector(s) does your organisation operate in??',
 'What sector(s) does your organisation operate in?',
 'Where is your organisation based? Select all that apply',
 'Where is your organisation based? Select all that apply [Other]',
 'Where is your company based? ',
 'Which theme of the True North report do you most identify with and could support activity around? (select all that apply)',
 'Which theme of the True North report do you most identify with and could support activity around? ',
 'Do you feel the True North report identified the key challenges and opportunities facing the region? ',
 'Are you interested in attending future True North events? ',
 'How would you like to be involved with the True North network (select all that apply)? [Other]',
 'How would you like to be involved with the True North

Write a list of all the columns we don't need, and then remove them (drop) from the dataframe

In [7]:
cols = ['Last Activity Date', 'Where is your company based? ']
data.drop(columns=cols, inplace=True)

In the following 4 code blocks this logic applies:
- Make a list of columns that are the same question and need merging. 
- Merge them together with priority according to the order of the list. 
- If an entry exists, use that value. Otherwise, go to the next column.
- Add this as a new, single column with a new title.
- Remove the old columns from the dataframe.

In [8]:
cols = ['What sector(s) does your organisation operate in? [Other]', 'What sector(s) does your organisation operate in??', 'What sector(s) does your organisation operate in?']
# cols are F,G,H
# data['sector'] = data[cols[0]].combine_first(data[cols[1]]).combine_first(data[cols[2]])

def combine_sectors(row):
    if row[cols[2]] == 'Other - please specify':
        return row[cols[0]]
    elif pd.notnull(row[cols[1]]):
        return row[cols[1]]
    elif pd.notnull(row[cols[2]]):
        return row[cols[2]]
    else:
        return None

data['sector'] = data.apply(combine_sectors, axis=1)
data.drop(columns=cols, inplace=True)

In [9]:
cols = ['Where is your organisation based? Select all that apply', 'Where is your organisation based? Select all that apply [Other]']
data['location'] = data[cols[0]].combine_first(data[cols[1]])
data.drop(columns=cols, inplace=True)

In [10]:
cols = ['Which theme of the True North report do you most identify with and could support activity around? (select all that apply)', 
        'Which theme of the True North report do you most identify with and could support activity around? ']
data['Which theme of the True North report do you most identify with and could support activity around?'] = data[cols[0]].combine_first(data[cols[1]])
data.drop(columns=cols, inplace=True)

In [11]:
cols = ['How would you like to be involved with the True North network (select all that apply)? [Other]',
        'How would you like to be involved with the True North network (select all that apply)?']
data['How would you like to be involved with the True North network?'] = data[cols[0]].combine_first(data[cols[1]])
data.drop(columns=cols, inplace=True)

Mapping the company sizes onto the standard ranges. Assuming there are no edge cases.

One company had both 1-10 and 11-50 as a company range. We've opted for the 10-49 range based on probability.

In [12]:
data['How many people work in your organisation?'].unique()

array([nan, '501 - 1,000', '11 - 50', '1,000+', '1 - 10', '51 - 100',
       '101 - 250', '251 - 500', '1 - 10; 11 - 50'], dtype=object)

In [13]:
mapper = {
    '1 - 10': '0-9', 
    '51 - 100': '50-249', 
    '11 - 50': '10-49', 
    '101 - 250': '50-249',
    '251 - 500': '250+', 
    '501 - 1,000': '250+', 
    '1 - 10; 11 - 50': '10-49', 
    '51 - 200': '50-249',
    '2 - 10': '0-9', 
    '501 - 1000': '250+', 
    '11 - 50 ': '10-49', 
    '1,000+': '1000+',
    '104 - 250': '50-249', 
    '51-200': '50-249', 
    '101-250': '50-249', 
    '2 - 11': '0-9', 
    'Self-employed': 'Self-Employed', 
    'Self employed': 'Self-Employed',
    'nan': ''
    }

data['company_size'] = data['How many people work in your organisation?'].replace(mapper)
data.drop(columns='How many people work in your organisation?', inplace=True)

Show the new, cleaner categories

In [14]:
data.company_size.unique()

array([nan, '250+', '10-49', '1000+', '0-9', '50-249'], dtype=object)

Sort the values by last_updated date, with oldest at the top.

In [15]:
data.sort_values(by='last_updated', inplace=True, ascending=False)
data.set_index('last_updated', inplace=True)

We found that some entries are likely to be from the same company. Currently these are:
- TUBR LTD; gettubr.com
- Sitehop; Sitehop.com

We will manually correct these, using the row with the actual company name (not url) as prioirity, and filling blanks with any entries (if present) from the other row.

In [16]:
# def merge_duplicate_companies(data, names):
#     '''
#         Names is a list of 2 company names to merge. 
#         First company name in the list is used as priority when merging.
#         Function assumes only 2 duplicate entries. Can be adjusted in future if we need to account for n duplicates.
#     '''
#     # reset the index to access data by a numeric index
#     data.reset_index(inplace=True)
    
#     # get the indices of the named rows
#     index_a = data.index[data['Company Name'] == names[0]]
#     index_b = data.index[data['Company Name'] == names[1]]

#     # combine the rows
#     combined_row = data.iloc[index_a].combine_first(data.iloc[index_b])

#     # insert the row into the dataframe at index_a 
#     data.loc[index_a] = combined_row

#     # drop index b as no longer needed, reset the index and drop the old one, set the index to "last_updated".
#     data = data.drop(index_b).reset_index(drop=True).set_index('last_updated')

#     print('Successfully merged rows and created the following row: \n', data.loc[data['Company Name'] == names[0]].to_csv())

#     return data

Find rows where there company name is duplicated, but not NA, and the first and last name of the contact are not duplicated.

Get the names of these companies

iterate through them

find the rows that match this company name

sort them by date descending

remove the all rows from `data` apart from the most recent one.

In [17]:
names = data[data['Company Name'].duplicated() & data['Company Name'].notna() & data['First Name'].duplicated() & data['Last Name'].duplicated()]['Company Name']
# data = merge_duplicate_companies(data, ['TUBR', 'gettubr.com'])
# data = merge_duplicate_companies(data, ['Sitehop', 'Sitehop'])
names.values
for name in names.values:
    dupe_rows = data[data['Company Name']==name].copy().reset_index()
    dupe_rows.sort_values(by='last_updated', ascending=False, inplace=True)
    rows_to_drop = dupe_rows.iloc[1:, :]
    data.drop(index=rows_to_drop['last_updated'], inplace=True)

Remove Brabners from the clean file

In [18]:
pattern = r'(?i)\b\w*brabners\w*\b'
data = data[~((data['Company Name'].str.contains(pattern, regex=True)) & (data['Company Name'] != 'NaN'))]

In [19]:
data.drop(columns=['First Name', 'Last Name'], inplace=True)

Write the data to a csv called 'true_north_may_2024_clean"

In [20]:
data.to_csv(os.path.join(WDIR, 'true-north/true_north_clean.csv'))